## Cálculo de cuántos números primos hay en un intervalo

In [4]:
def prime_count_vanilla_range(range_from: int, range_til: int):
    """ Returns the number of found prime numbers using range"""
    prime_count = 0
    range_from = range_from if range_from >= 2 else 2
    for num in range(range_from, range_til + 1):
        for divnum in range(2, num):
            if ((num % divnum) == 0):
                break
            else:
                prime_count += 1
    return prime_count

### Option 1: Calculate each number sequentially (spoiler: this is really inefficient)

In [5]:
import time

numbers = [(1, 60000)]

start_time = time.time()

#  Truqillo para pasarle varios parámetros a una función

for pair in numbers:
    prime_sum = prime_count_vanilla_range(*pair)

end_time = time.time()

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 83.661500s


In [14]:
numbers = [(1, 60000)]


for pair in numbers:
    %time prime_sum = prime_count_vanilla_range(*pair)
print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')


for pair in numbers:
    %timeit -r1 prime_sum = prime_count_vanilla_range(*pair)
print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')

CPU times: user 1min 23s, sys: 31.9 ms, total: 1min 23s
Wall time: 1min 24s
Números primos encontrados en el intervalo [(1, 60000)]: 172199035
1min 24s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Números primos encontrados en el intervalo [(1, 60000)]: 172199035


El módulo time en Python se encarga de medir el tiempo de CPU consumido por un proceso, considerando diversos factores como tiempos de espera y otros elementos del sistema que pueden afectar la ejecución del código. Esta medición no se limita exclusivamente al tiempo directamente atribuible al código en ejecución.

En el contexto de IPython, se introducen funciones mágicas como %time y %timeit para medir el tiempo de manera más sofisticada, minimizando el impacto de factores externos. %time mide el tiempo de una única ejecución, mientras que %timeit realiza múltiples ejecuciones y calcula el mejor tiempo promedio. Es importante tener en cuenta que %timeit puede introducir un tiempo adicional debido a la configuración y ejecución de múltiples repeticiones para obtener un promedio más preciso. Estas herramientas son útiles para evaluar el rendimiento del código de una manera más robusta y precisa.

### A Python Implementation Using Numba: The fastest

In [9]:
import time
from numba import njit

# The same implementation as prime_count_vanilla_range w/ @njit before
@njit
def prime_count_numba(range_from: int, range_til: int):
    """ Returns the number of found prime numbers using range"""
    prime_count = 0
    range_from = range_from if range_from >= 2 else 2
    for num in range(range_from, range_til + 1):
        for divnum in range(2, num):
            if ((num % divnum) == 0):
                break
            else:
                prime_count += 1
    return prime_count

numbers = [(1, 60000)]

start_time = time.time()

#  Truqillo para pasarle varios parámetros a una función
for pair in numbers:
    prime_sum = prime_count_numba(*pair)

end_time = time.time()

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 2.787184s


La disparidad en los tiempos de ejecución se destaca al comparar el rendimiento de la función sin la optimización de Numba (njit) y con ella. Sin njit, el tiempo total para encontrar los resultados es de 38.9 s segundos, mientras que con njit, el tiempo mejora significativamente a 2.787184s segundos.

La necesidad de llamar dos veces a la función prime_count_numba se explica por el proceso de compilación llevado a cabo por Numba. En la primera invocación, Numba realiza la compilación just-in-time (JIT), que implica un tiempo adicional para traducir el código a una forma optimizada. Una vez que esta compilación ha ocurrido en la primera llamada, la función queda optimizada. En la segunda invocación y subsiguientes, se elimina el impacto del tiempo de compilación, ya que la función ya está optimizada, resultando en una ejecución más rápida. En resumen, la segunda llamada evita el tiempo adicional de compilación asociado con la primera invocación, mejorando así el rendimiento general de la función.

### Parallel 1: Using multiprocessing to parallelise

In [10]:
import multiprocessing
import time

numbers = [(1, 60000)]
intervals = [(1, 10000), (10001, 20000), (20001, 30000), (30001, 40000), (40001, 50000), (50001, 60000)]

start_time = time.time()

# Execute asynchronously with multi processing
# Using starmap as the function has several parameters

with multiprocessing.Pool(processes=6) as pool:
    result = pool.starmap(prime_count_vanilla_range, intervals)
                     
end_time = time.time()

print(list(result))

prime_sum = sum(result)

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

[5766453, 15483380, 24562880, 33560052, 41914240, 50912030]
Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 28.923095s


### Parallel 2: Using concurrent.futures (map) to parallelise

In [11]:
import concurrent.futures
import time

numbers = [(1, 60000)]
intervals = [(1, 10000), (10001, 20000), (20001, 30000), (30001, 40000), (40001, 50000), (50001, 60000)]

# La función necesita varios parámetros 
# Truqillo para pasarle varios parámetros a una función haciendo otra función como intermedia

def run_heavy_function(params):
    return prime_count_vanilla_range(*params)

start_time = time.time()
# Execute asynchronously with multi processing

with concurrent.futures.ProcessPoolExecutor(max_workers=6) as executor:
    resul = executor.map(run_heavy_function, intervals)

end_time = time.time()

resultados = list(resul)

print(resultados)

prime_sum = sum(resultados)

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

[5766453, 15483380, 24562880, 33560052, 41914240, 50912030]
Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 26.079178s


### Parallel 3: Using concurrent.futures (submit) to parallelise

In [12]:
import concurrent.futures
import time
from boltons import iterutils as ite

numbers = [(1, 60000)]
#intervals = [(1, 10000), (10001, 20000), (20001, 30000), (30001, 40000), (40001, 50000), (50001, 60000)]

start_time = time.time()
# Execute asynchronously with multi processing

with concurrent.futures.ProcessPoolExecutor(max_workers=6) as exe:
    jobs = []
    for frm, to in ite.chunk_ranges(input_size=60000, chunk_size=10000):
        job = exe.submit(prime_count_vanilla_range, frm, to)
        jobs.append(job)

end_time = time.time()

print([job.result() for job in jobs])

prime_sum = sum(job.result() for job in jobs)

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

ModuleNotFoundError: No module named 'boltons'

A pesar de la subdivisión del trabajo en seis intervalos para la búsqueda de números primos en paralelo, la mejora observada es de aproximadamente 5 veces en lugar de 6. Esta discrepancia se explica al considerar que no solo se están ejecutando simultáneamente seis funciones, sino que también se están dividiendo seis intervalos de trabajo asignados a seis núcleos. Esto implica una gestión y comunicación adicionales entre los procesos, lo que introduce un tiempo adicional en el procesamiento.

Para abordar esta situación haciendo uso de Numba, se sugiere incorporar una sección de código dependiente de Numba después de haber ejecutado previamente los seis intervalos en paralelo. En este punto, se podría integrar la función `prime_count_numba()` para optimizar el proceso. Al revisar el código previo, se destaca la oportunidad de aprovechar nuevamente la función `prime_count_numba()` e implementarla en el código para obtener beneficios de optimización.

In [13]:
from numba import njit
import concurrent.futures

@njit
def prime_count_numba(range_from: int, range_til: int):
    """ Returns the number of found prime numbers using range"""
    prime_count = 0
    range_from = range_from if range_from >= 2 else 2
    for num in range(range_from, range_til + 1):
        for divnum in range(2, num):
            if ((num % divnum) == 0):
                break
            else:
                prime_count += 1
    return prime_count

# Función para ejecutar en paralelo
def run_numba_parallel(params):
    return prime_count_numba(*params)

numbers = [(1, 60000)]
intervals = [(1, 10000), (10001, 20000), (20001, 30000), (30001, 40000), (40001, 50000), (50001, 60000)]

start_time = time.time()

with concurrent.futures.ProcessPoolExecutor(max_workers=6) as executor:
    result = executor.map(run_numba_parallel, intervals)

end_time = time.time()

results_list = list(result)
prime_sum = sum(results_list)

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 2.002522s


En este escenario, la utilización de `@njit` ha logrado reducir el tiempo de compilación mucho, al implementarlo en cada una de las ejecuciones en paralelo que estamos llevando a cabo. Además, la mejora es considerablemente significativa cuando se compara con el tiempo que requería la ejecución de la función original.